In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import time
from timeit import default_timer as timer
import torch
from torch.autograd import Variable

import numpy as np
import pickle
import pathlib

if torch.cuda.is_available():
    print('cuda available')
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
else:
    print('cuda not available')
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor

cuda available


In [2]:
from core.SimpleConvNet import SimpleConvNet
from core.GraphConvNet2 import GraphConvNet2
from core.DataEmbeddingGraph import DataEmbeddingGraph
from util.mnist_data_loader import get_train_set, get_test_set
from util.plot_embedding import plot_embedding, plot_embedding_subplot
from util.draw_random_subset import draw_random_subset

In [3]:
def save_checkpoint(state, filename):
    torch.save(state, filename)

In [4]:
task_parameters = {}
task_parameters['net_type'] = 'graph_net'
task_parameters['reduction_method'] = 'preprocessed_tsne'
task_parameters['loss_function'] = 'pairwise_loss'
task_parameters['n_components'] = 2

net_parameters = {}
net_parameters['n_components'] = task_parameters['n_components']
net_parameters['D'] = 784 # input dimension
net_parameters['H'] = 50 # number of hidden units
net_parameters['L'] = 10 # number of hidden layers
net_parameters['n_channels'] = 1 # for conv net
net_parameters['n_units_1'] = net_parameters['n_units_2'] = net_parameters['H'] # for conv net

In [5]:
if task_parameters['net_type'] == 'graph_net':
    net = GraphConvNet2(net_parameters)
elif task_parameters['net_type'] == 'conv_net':
    net = SimpleConvNet(net_parameters)
if torch.cuda.is_available():
    net.cuda()
print(net)

GraphConvNet2(
  (gnn_cells): ModuleList(
    (0): GraphConvNetCell(
      (Ui1): Linear(in_features=784, out_features=50, bias=False)
      (Uj1): Linear(in_features=784, out_features=50, bias=False)
      (Vi1): Linear(in_features=784, out_features=50, bias=False)
      (Vj1): Linear(in_features=784, out_features=50, bias=False)
      (Ui2): Linear(in_features=50, out_features=50, bias=False)
      (Uj2): Linear(in_features=50, out_features=50, bias=False)
      (Vi2): Linear(in_features=50, out_features=50, bias=False)
      (Vj2): Linear(in_features=50, out_features=50, bias=False)
      (bn1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (R): Linear(in_features=784, out_features=50, bias=False)
    )
    (1): GraphConvNetCell(
      (Ui1): Linear(in_features=50, out_features=50, bias=False)
      (Uj1): Linear(in_features=50, out_features=50, bias=F

In [6]:
# optimization parameters
opt_parameters = {}
opt_parameters['learning_rate'] = 0.00075   # ADAM
opt_parameters['max_iters'] = 200   
opt_parameters['batch_iters'] = 10
opt_parameters['max_train_size'] = 20000   
opt_parameters['save_flag'] = True

if 2==1: # fast debugging
    opt_parameters['max_iters'] = 25 
    opt_parameters['batch_iters'] = 5
    opt_parameters['save_flag'] = False

opt_parameters['decay_rate'] = 1.25   

In [7]:
if opt_parameters['save_flag']: 
    checkpoint_interval = opt_parameters['max_iters']/5
    checkpoint_root = 'results/mnist_' + task_parameters['reduction_method'] + '1/'
    pathlib.Path(checkpoint_root).mkdir(exist_ok=True) # create the directory if it doesn't exist

In [8]:
# Pre-processed train data
parent_dir = os.path.abspath('..')
filename = parent_dir + '/data/set_20000_mnist_tsne.pkl'
with open(filename, 'rb') as f:
    [inputs, labels, X_emb] = pickle.load(f)

In [9]:
all_train_data = []
num_train_samples = 0
while num_train_samples <= opt_parameters['max_train_size']:
    # Draw a random training batch of variable size
    num_samples = np.random.randint(200, 500)
    inputs_subset = inputs[num_train_samples:num_train_samples+num_samples]
    labels_subset = labels[num_train_samples:num_train_samples+num_samples]
    X_emb_subset = X_emb[num_train_samples:num_train_samples+num_samples]
    
    # Package into graph block
    G = DataEmbeddingGraph(inputs_subset, labels_subset, 'spectral')
    G.target = X_emb_subset
    
    all_train_data.append(G)
    num_train_samples += num_samples  

In [10]:
all_train_data = all_train_data[:len(all_train_data)-1]

In [11]:
dataset_sizes = [len(G.labels) for G in all_train_data]
print("Dataset sizes = {}".format(dataset_sizes))
print("Total number of samples = {}".format(sum(dataset_sizes)))

Dataset sizes = [465, 278, 393, 452, 470, 425, 404, 442, 386, 405, 417, 480, 494, 304, 334, 347, 200, 279, 252, 400, 362, 240, 488, 464, 387, 211, 366, 374, 382, 329, 278, 425, 470, 479, 317, 209, 495, 385, 397, 233, 414, 303, 218, 396, 471, 482, 497, 382, 242, 444, 247, 237, 271, 277]
Total number of samples = 19799


In [12]:
# Optimization parameters
learning_rate = opt_parameters['learning_rate']
max_iters = opt_parameters['max_iters']
batch_iters = opt_parameters['batch_iters']
decay_rate = opt_parameters['decay_rate']

# Optimizer
global_lr = learning_rate
global_step = 0
lr = learning_rate
optimizer = net.update(lr) 

# Statistics
t_start = time.time()
t_start_total = time.time()
average_loss_old = 1e10
running_loss = 0.0
running_total = 0
    
for iteration in range(max_iters):
    
    for G in all_train_data:
        # Forward pass
        y_pred = net.forward(G)

        # Target embedding matrix
        y_true = G.target
        y_true = Variable(torch.FloatTensor(y_true).type(dtypeFloat) , requires_grad=False) 

        # Compute overall loss 
        if task_parameters['loss_function'] == 'pairwise_loss':
            loss = net.pairwise_loss(y_pred, y_true, G.adj_matrix)
        elif task_parameters['loss_function'] == 'composite_loss':
            loss1 = net.loss(y_pred, y_true)
            loss2 = net.pairwise_loss(y_pred, y_true, G.adj_matrix)
            loss = 0.5*loss1 + 0.5*loss2
            
        loss_train = loss.data[0]
        running_loss += loss_train
        running_total += 1

        # Backprop
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # learning rate, print results
    if not iteration%batch_iters:

        # time
        t_stop = time.time() - t_start
        t_start = time.time()

        # update learning rate 
        average_loss = running_loss/ running_total
        if average_loss > 0.99* average_loss_old:
            lr /= decay_rate
        average_loss_old = average_loss
        optimizer = net.update_learning_rate(optimizer, lr)
        running_loss = 0.0
        running_total = 0

        # print results
        print('\niteration= %d, loss(%diter)= %.8f, lr= %.8f, time(%diter)= %.2f' %
              (iteration, batch_iters, average_loss, lr, batch_iters, t_stop))
            
        # validate on test set
        #######################
    
    if opt_parameters['save_flag'] and not (iteration+1)%checkpoint_interval:
        filename = checkpoint_root + task_parameters['net_type'] + str(int((iteration+1)/checkpoint_interval)) + '.pkl'
        save_checkpoint({
            'state_dict': net.state_dict(),
            'optimizer' : optimizer.state_dict(),
        }, filename)

/home/leowyaoyang/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



iteration= 0, loss(10iter)= 2225.87451172, lr= 0.00075000, time(10iter)= 14.62

iteration= 10, loss(10iter)= 260.56207275, lr= 0.00075000, time(10iter)= 133.71

iteration= 20, loss(10iter)= 60.03235245, lr= 0.00075000, time(10iter)= 134.26

iteration= 30, loss(10iter)= 33.89888000, lr= 0.00075000, time(10iter)= 134.15

iteration= 40, loss(10iter)= 24.02558517, lr= 0.00075000, time(10iter)= 134.74

iteration= 50, loss(10iter)= 21.25079536, lr= 0.00075000, time(10iter)= 138.86

iteration= 60, loss(10iter)= 19.56133270, lr= 0.00075000, time(10iter)= 139.35

iteration= 70, loss(10iter)= 12.43511009, lr= 0.00075000, time(10iter)= 139.35

iteration= 80, loss(10iter)= 16.04421234, lr= 0.00060000, time(10iter)= 139.06

iteration= 90, loss(10iter)= 11.13127613, lr= 0.00060000, time(10iter)= 139.41

iteration= 100, loss(10iter)= 8.27100945, lr= 0.00060000, time(10iter)= 139.23

iteration= 110, loss(10iter)= 8.26477814, lr= 0.00048000, time(10iter)= 139.82

iteration= 120, loss(10iter)= 11.90534

## On test data

In [13]:
inputs, labels = draw_random_subset(test_data, num_samples=500)
G = DataEmbeddingGraph(inputs, labels, task_parameters['reduction_method'])
net_time_start = timer()
if torch.cuda.is_available():   
    y_pred = net.forward(G).cpu().detach().numpy()
else:    
    y_pred = net.forward(G).detach().numpy()
net_time_end = timer()

f, axarr = plt.subplots(1, 2, sharex='col', sharey='row', figsize=(10, 3), dpi=150)
reduction_title = task_parameters['reduction_method'] + "\n time elapsed = {:.2f}s".format(G.time_to_compute) 
cnn_title = "Graph CNN" + "\n time elapsed = {:.2f}s".format(net_time_end - net_time_start) 
plot_embedding_subplot(axarr[0], G.target, G.labels.numpy(), reduction_title)
plot_embedding_subplot(axarr[1], y_pred, G.labels.numpy(), cnn_title)

NameError: name 'test_data' is not defined

In [14]:
if opt_parameters['save_flag']:
    metadata_filename = checkpoint_root + "experiment_metadata.txt"
    with open(metadata_filename, 'w') as f:
        f.write('-----------------------\n')
        f.write('\n'.join(["%s = %s" % (k,v) for k,v in task_parameters.items()]))
        
        f.write('\n-----------------------\n')
        f.write('\n'.join(["%s = %s" % (k,v) for k,v in net_parameters.items()]))
        
        f.write('\n-----------------------\n')
        f.write('\n'.join(["%s = %s" % (k,v) for k,v in opt_parameters.items()]))